In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error


# Notebook with EDA and process to Model Selection

## 1) EDA

The goal of the EDA is:
- look for any missing values in the data set
- look for eventual error measurements to be deleted or corrected (we dont necessarily pay very much attention to outliers because they are going to be many for a data set of this size)
- look which predictor variables might be useful to predict bike count 

Before starting the EDA, we remind the reader that the target variable (ie what we are trying to predict is the variable `log_bike_count`

We first import the original data, set check for missing values and error measurements and then merge it with the external and check no data was lost in the process

In [2]:
pd.set_option("display.precision", 2)  # set precision of outputs of our df

# read the data
df_original = pd.read_parquet(Path("data") / "train.parquet")
df_original.head()


,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,counter_technical_id,latitude,longitude,log_bike_count
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,Y2H15027244,48.85,2.38,0.00
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,Y2H15027244,48.85,2.38,0.69
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,Y2H15027244,48.85,2.38,0.00
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,Y2H15027244,48.85,2.38,1.61
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,Y2H15027244,48.85,2.38,2.30


In [3]:
df_original.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 455163 entries, 48321 to 928462
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   counter_id                 455163 non-null  category      
 1   counter_name               455163 non-null  category      
 2   site_id                    455163 non-null  int64         
 3   site_name                  455163 non-null  category      
 4   bike_count                 455163 non-null  float64       
 5   date                       455163 non-null  datetime64[ns]
 6   counter_installation_date  455163 non-null  datetime64[ns]
 7   counter_technical_id       455163 non-null  category      
 8   latitude                   455163 non-null  float64       
 9   longitude                  455163 non-null  float64       
 10  log_bike_count             455163 non-null  float64       
dtypes: category(4), datetime64[ns](2), float64(4), i

In [4]:
df_original.describe()


,site_id,bike_count,latitude,longitude,log_bike_count
count,4.55e+05,455163.00,455163.00,455163.00,455163.00
mean,1.05e+08,59.48,48.85,2.35,3.05
std,3.16e+07,87.13,0.02,0.04,1.68
min,1.00e+08,0.00,48.83,2.27,0.00
25%,1.00e+08,4.00,48.84,2.31,1.61
50%,1.00e+08,28.00,48.85,2.35,3.37
75%,1.00e+08,79.00,48.86,2.38,4.38
max,3.00e+08,1275.00,48.89,2.41,7.15


There are 455 163 observations in the training data set and 9 predictor variables. However some of them like `site_name` and `site_id`, or `counter_name` and `counter_id` are redundant and both should not be included in the model. Also,  `site_id` is an int so we should remember it is a categorical variable if we want to use it in our model. <br>
Finally, there are no missing values. 
Let us now perform the merge and check how is the df.

In [5]:
file_path = Path("data") / "external_data.csv"
df_ext = pd.read_csv(file_path, parse_dates=["date"])

# merge on date to obtain the full data set.

df_merge_ext = pd.merge_asof(df_original.sort_values(
    "date"), df_ext.sort_values("date"), on="date")


In [6]:
df_merge_ext.head()


,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,counter_technical_id,latitude,longitude,...,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4,hol_scol,hol_bank,quarantine1,quarantine2
0,100056332-104056332,Pont de Bercy SO-NE,100056332,Pont de Bercy,0.0,2020-09-01 01:00:00,2019-12-11,Y2H19070378,48.84,2.38,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False
1,100047547-104047547,6 rue Julia Bartet NE-SO,100047547,6 rue Julia Bartet,4.0,2020-09-01 01:00:00,2018-11-28,Y2H18086323,48.83,2.30,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False
2,100047547-103047547,6 rue Julia Bartet SO-NE,100047547,6 rue Julia Bartet,2.0,2020-09-01 01:00:00,2018-11-28,Y2H18086323,48.83,2.30,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False
3,100057380-103057380,Totem Cours la Reine O-E,100057380,Totem Cours la Reine,0.0,2020-09-01 01:00:00,2020-02-11,YTH19111509,48.86,2.31,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False
4,100047548-103047548,Face au 25 quai de l'Oise NE-SO,100047548,Face au 25 quai de l'Oise,2.0,2020-09-01 01:00:00,2018-11-28,Y2H18086324,48.89,2.38,...,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,False


From the head, we already see that some columns seem to have a lot of missing values. We investigate this further. We also check first if there are duplicate rows.

In [7]:
df_merge_ext.duplicated().sum()


0

There are no duplicate columns.

In [8]:
df_merge_ext.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 455163 entries, 0 to 455162
Data columns (total 75 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   counter_id                 455163 non-null  category      
 1   counter_name               455163 non-null  category      
 2   site_id                    455163 non-null  int64         
 3   site_name                  455163 non-null  category      
 4   bike_count                 455163 non-null  float64       
 5   date                       455163 non-null  datetime64[ns]
 6   counter_installation_date  455163 non-null  datetime64[ns]
 7   counter_technical_id       455163 non-null  category      
 8   latitude                   455163 non-null  float64       
 9   longitude                  455163 non-null  float64       
 10  log_bike_count             455163 non-null  float64       
 11  Unnamed: 0.1               455163 non-null  int64   

We have 75 columns, too many to see the number of null values so we print out only the columns with missing values.

In [9]:
print(df_merge_ext.isnull().sum()[df_merge_ext.isnull().sum() > 0])
print(
    f"there are {df_merge_ext.isnull().sum()[df_merge_ext.isnull().sum()>0].shape[0]} columns with at least 1 missing values ")


w1              996
w2             1488
n             20682
nbas            834
hbas          61548
cl            58344
cm           200413
ch           237391
niv_bar      455163
geop         455163
tend24         1332
tn12         341684
tn24         455163
tx12         341684
tx24         455163
tminsol      455163
sw           455163
tw           455163
raf10          1332
etat_sol       7980
ht_neige       7184
ssfrai        63374
perssfrai     63374
rr1             840
rr3             504
rr6            1500
rr12           1836
rr24           1836
phenspe1     455163
phenspe2     455163
phenspe3     455163
phenspe4     455163
nnuage1       60894
ctype1       110472
hnuage1       61884
nnuage2      221760
ctype2       257898
hnuage2      221760
nnuage3      371079
ctype3       392331
hnuage3      371079
nnuage4      449829
ctype4       443163
hnuage4      449829
dtype: int64
there are 44 columns with at least 1 missing values 


Hence, we see that most columns have missing values and from the output, some columns have no non NA values! <br>
We drop the columns that have more than 10% missing values first as we are unlikely to be able to get interesting predictions with that little amount of data.


In [10]:
n_rows = df_merge_ext.shape[0]

columns_to_drop = (df_merge_ext.isnull().sum()/n_rows)[(df_merge_ext.isnull().sum()/n_rows) > 0.1].index

df_merge_ext = df_merge_ext.drop(columns=columns_to_drop)



Let us check now the missing values again.

In [11]:
df_merge_ext.reset_index()
#df_merge_ext[df_merge_ext.isnull().sum()>0]
np.sum(df_merge_ext.isnull().sum()>0)

13

There are 13 columns with missing values. How are these values missing, are they MCAR, MAR, or is there any pattern in the missing values?


graph qui plot les counts 

## 2) Model Selection

In [12]:
def _merge_external_data(X):
    file_path = Path("data") / "external_data.csv"
    df_ext = pd.read_csv(file_path, parse_dates=["date"])

    X = X.copy()
    # When using merge_asof left frame need to be sorted
    X["orig_index"] = np.arange(X.shape[0])

    X = pd.merge_asof(  # , "nbas" , "raf10"
        X.sort_values("date"), df_ext[["date", "hol_bank", "hol_scol", "quarantine1", "quarantine2", "t", "rr1", "u", "nbas", "raf10"]].sort_values("date").dropna(), on="date")  # , direction="nearest"
    # Sort back to the original order
    X = X.sort_values("orig_index")
    del X["orig_index"]
    return X


def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "weekday"] = X["date"].dt.weekday
    X.loc[:, "hour"] = X["date"].dt.hour

    X.loc[:, "weekend"] = X["weekday"] > 4

    X['sin_hours'] = np.sin(2*np.pi*X["hour"]/24)
    X['cos_hours'] = np.cos(2*np.pi*X["hour"]/24)

    X['sin_mnth'] = np.sin(2*np.pi*X["month"]/12)
    X['cos_mnth'] = np.cos(2*np.pi*X["month"]/12)

    return X.drop(columns=["date"])


First we create a function to easily get the different features we want to test.
We get:
- the features that do not need to be preprocessed 
- the categorical features that need 1-0 encoding

In [13]:
# function to get the features that do not need to be processed
def get_passthrough(date, list_of_temp):
    """function to get the features that will not be transformed at the prepocessing stage

    Args:
        date (str): "both_date": select all the date features                   
                    "original_date": selecte date without sin transformed
                    "transformed_date": select date with sin-cos transformation

        list_of_temp (list): list of features (no date) that will not be transformed

    Returns:
        _type_: features to not be transformed  
    """
    pass_through_cols = []
    if "both_date" == date:
        pass_through_cols = ["hour", "day", "weekday", "month",
                             "year", "sin_hours", "cos_hours", "sin_mnth", "cos_mnth"]

    if "original_date" == date:
        pass_through_cols = ["hour", "day", "weekday", "month", "year"]

    if "transformed_date" == date:
        pass_through_cols = ["sin_hours", "cos_hours",
                             "sin_mnth", "cos_mnth", "year", "weekday"]

    for el in list_of_temp:
        pass_through_cols.append(el)

    return pass_through_cols


In [14]:
def get_estimator(pass_through_cols, categorical_cols, regressor=XGBRegressor()):

    # define the encoders
    categorical_encoder = OneHotEncoder(handle_unknown="ignore")
    date_encoder = FunctionTransformer(_encode_dates)

    # define the transformation of data before using regressor
    preprocessor = ColumnTransformer(
        [
            ("cat", categorical_encoder, categorical_cols),
            # ("std_scaler", StandardScaler(), numerical_cols),
            ("passthrough", "passthrough", pass_through_cols)
        ],
    )

    pipe = make_pipeline(
        FunctionTransformer(_merge_external_data, validate=False),
        date_encoder,
        preprocessor,
        regressor,
    )

    return pipe


In [15]:
# test
a = ["sin_hours", "cos_hours", "sin_mnth", "cos_mnth"]
b = ["counter_name", "site_name", "weekday", "weekend"]
model = get_estimator(a, b, regressor=Ridge())
model


Pipeline(steps=[('functiontransformer-1',
                 FunctionTransformer(func=<function _merge_external_data at 0x7fcd26503dc0>)),
                ('functiontransformer-2',
                 FunctionTransformer(func=<function _encode_dates at 0x7fcd26503e50>)),
                ('columntransformer',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['counter_name', 'site_name',
                                                   'weekday', 'weekend']),
                                                 ('passthrough', 'passthrough',
                                                  ['sin_hours', 'cos_hours',
                                                   'sin_mnth', 'cos_mnth'])])),
                ('ridge', Ridge())])

In [16]:
import problem

X_train, y_train = problem.get_train_data()
X_test, y_test = problem.get_test_data()


model.fit(X_train, y_train)


Pipeline(steps=[('functiontransformer-1',
                 FunctionTransformer(func=<function _merge_external_data at 0x7fcd26503dc0>)),
                ('functiontransformer-2',
                 FunctionTransformer(func=<function _encode_dates at 0x7fcd26503e50>)),
                ('columntransformer',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['counter_name', 'site_name',
                                                   'weekday', 'weekend']),
                                                 ('passthrough', 'passthrough',
                                                  ['sin_hours', 'cos_hours',
                                                   'sin_mnth', 'cos_mnth'])])),
                ('ridge', Ridge())])

In [17]:
from sklearn.metrics import mean_squared_error

print(
    f"Train set, RMSE={mean_squared_error(y_train, model.predict(X_train), squared=False):.2f}"
)
print(
    f"Test set, RMSE={mean_squared_error(y_test, model.predict(X_test), squared=False):.2f}"
)


Train set, RMSE=0.97
Test set, RMSE=0.91


In [18]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score



In [24]:
def test_model(pass_throughs_col, categorical_cols, regressor = XGBRegressor(max_depth = 4, subsample = 0.8)):
    """test a model given the features and the regressor and output the prediction on the test data set and the scores on train and test

    Args:
        pass_throughs_col (list): list of features not to be transform during pre-processing step
        categorical_cols (list): columns to be one hot encoded
        regressor (regressor to use in our model, optional): scikit compatible function. Defaults to XGBRegressor().

    Returns:
        array,float,float: the prediction on the test set, the score on the training set, the score on the testing set
    """
    X_train, y_train = problem.get_train_data()
    X_test, y_test = problem.get_test_data()    
    model = get_estimator(pass_throughs_col,categorical_cols,regressor)
    model.fit(X_train, y_train)
    
    cv = TimeSeriesSplit(n_splits=6)

    scores = cross_val_score(
        model, X_train, y_train, cv=cv, scoring="neg_root_mean_squared_error"
    )
    print("RMSE: ", scores)
    print(f"RMSE (all folds): {-scores.mean():.3} ± {(-scores).std():.3}")

    #return model.predict(X_test), mean_squared_error(y_train, model.predict(X_train)), mean_squared_error(y_test, model.predict(X_test), squared=False)
    return model.predict(X_test), round(mean_squared_error(y_train, model.predict(X_train)),3), \
        round(-scores.mean(),3), round(-scores.std(),3) , \
        round(mean_squared_error(y_test, model.predict(X_test), squared=False),3)

    

In [ ]:
test_model(a,b)

Now that we have our functions to test different models quickly, we are going to try some different models using linear regression and Xgboost and compare the RMSE. Because XGboost tends to overfitt, we are going to limit the complexity of our model to an arbitrary threshold.<br>
We are going to test:
- the minimum model with date not transformed and no external data added
- min model with date transformed (we dont use both dates as it is redundant)
- min model with weather 
- min model with quarantine 
- min model with holidays

Then we are going to try different combination of the above models.

In [21]:
list_of_pass = []
